In [34]:
import os
from IPython.display import Image, display, Markdown
from openai import OpenAI
import openai
import json


In [2]:
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [28]:
def generate_proj(details, model="gpt-5.1"):
    system_prompt = f"""
You are a professional resume builder who specializes in crafting crisp, exciting project descriptions for resumes.

Your first task is to ask the user to describe their project in simple terms
(what it does, the problem it solves, the tech used, and key features).
if the user does not provides the project tiltle u should genearte that from the project description.

After receiving the description, you must:
1.CREATE a strong, professional, recruiter-friendly project title.
2.Write an energetic, crisp project description (4–6 lines).
3.Avoid generic statements.
4.Generate the tech stack

Maintain a thrilling, high-energy tone.

Return the output *strictly in JSON* using this format

{{
    "project_title": "...",
    "project_description": "...",
    "tech_stack": ["...", "..."],
    "tokens": <int>,
    "tone": "energetic",
    "sources": ["user_input"],
    "confidence": <float between 0 and 1>
}}

"""
    user_prompt = details
    messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt}
  ]

  #3. Call OpenAI
    response = openai.chat.completions.create(model=model, messages=messages,temperature=0.0)

  #4. Print the results
    content = response.choices[0].message.content

    try:
      data = json.loads(content)
    except:
      data={"about":content,"highlights":[],"confidence":0.5}
    return content


In [42]:
user_project_details ="""
 I build a project on cat vs dog classification using CNNs.
 """

In [38]:
generated_project = generate_proj(user_project_details)
print(generated_project)

{
  "project_title": "Intelligent Cat vs Dog Image Classifier using CNNs",
  "project_description": "Built a convolutional neural network that automatically distinguishes cat images from dog images with high accuracy. Designed and trained a deep CNN pipeline including convolution, pooling, and fully connected layers for robust feature extraction. Implemented data preprocessing and augmentation to handle noisy, varied images and reduce overfitting. Evaluated performance using validation metrics and iteratively tuned hyperparameters to boost model accuracy and generalization.",
  "tech_stack": [
    "Python",
    "TensorFlow or PyTorch",
    "Keras (if used with TensorFlow)",
    "NumPy",
    "Pandas",
    "OpenCV or Pillow (PIL)",
    "Matplotlib / Seaborn",
    "Jupyter Notebook or Google Colab"
  ],
  "tokens": 132,
  "tone": "energetic",
  "sources": ["user_input"],
  "confidence": 0.86
}


In [39]:
def validate_project(project_details_generated,project_details_original,model="gpt-5.1"):
  system_prompt = f"""
            You are a strict resume project quality validator. You will receive:
            1) the generated project JSON (fields: project_title, project_description, tech_stack, tokens, tone, sources, confidence)
            2) the original user details used to generate the project entry

            Tasks:
            - Verify factual consistency: every project detail (title, description, tech_stack) must be fully supported by the original details.
             If any element is not supported, mark it as a hallucination.
            - Verify required fields: project_title, project_description, and tech_stack must be present and non-empty.
            - Verify tone: the description must be energetic, specific, and not generic.
            - Verify structure:
                1. project_title must be concise and meaningful
                2. project_description must be 4–8 lines (±2 lines tolerance)
                3. tech_stack must contain only technologies mentioned in user input
            - Make sure you perform a very critical evaluation of the write-up.

            Produce a strict JSON response only in this format:

            {{
              "decision": "go" or "no-go",
              "score": <float between 0 and 1>,
              "reasoning": ["..."] retain original reasoning content
              "checks": {{
                "length_ok": {{true/false}},
                "tone_specific": {{true/false}},
                "no_hallucinations": {{true/false}},
                "structure_valid": {{true/false}},
                "tech_stack_valid": {{true/false}}
              }}
            }}

              }}"""

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": json.dumps({
          "generated":project_details_generated,
          "original":project_details_original
      })} ]

  #3. Call OpenAI
  response = openai.chat.completions.create(model=model, messages=messages,temperature=0.0,max_completion_tokens=400)

  #4. Print the results
  content = response.choices[0].message.content

  try:
    result = json.loads(content)
  except Exception:
    result ={
        "decison": "no-go",
        "score": 0.0,
        "reasons": ["validator failed to parse model output"],
        "checks": {"length_ok": False,
                "uses_Specifies": False,
                "no_hallicunations:": False},

    }
  return result

In [40]:
validate = validate_project(generated_project,user_project_details)
print(validate)

{'decision': 'no-go', 'score': 0.32, 'reasoning': ["The original input only states: 'I build a project on cat vs dog classification using CNNs.' No other details are provided.", "The generated title adds 'Intelligent' and 'Image Classifier using CNNs' which is mostly inferable, but still slightly embellished beyond the minimal original statement.", 'The description introduces multiple specific implementation details (deep CNN pipeline structure, data preprocessing and augmentation, handling noisy images, validation metrics, hyperparameter tuning, focus on generalization) that are not mentioned or implied in the original input. These are standard assumptions but not supported facts, so they count as hallucinations.', 'The tech_stack lists many tools and libraries (Python, TensorFlow or PyTorch, Keras, NumPy, Pandas, OpenCV or Pillow, Matplotlib/Seaborn, Jupyter/Colab) that are not mentioned anywhere in the original input, and thus are hallucinated.', 'Required fields (project_title, pro

In [ ]:
# #Built and optimized deep learning models on the Fashion MNIST dataset.
#  Experimented with activation functions, network depth, filter dimensions, and dropout regularization.
#   Compared ANN vs CNN performance to uncover insights in image recognition.

In [ ]:
# Cats vs Dogs Image Classification using CNNs Built a CNN-based model to classify cats
# and dogs from color images with high breed and pose variation. Tuned convolutional layers, filters,
#  and pooling methods to strengthen spatial learning.
#  Enhanced generalization using dropout and image augmentation for regularization.